In [ ]:
import torch
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

In [ ]:
X, Y = datasets.load_iris(return_X_y=True)
print(X)
le=LabelEncoder()
Y=le.fit_transform(Y)
print(Y)
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, train_size=0.8)

X_train, X_test, Y_train, Y_test = torch.tensor(X_train, dtype=torch.float32),torch.tensor(X_test, dtype=torch.float32),torch.tensor(Y_train, dtype=torch.long),torch.tensor(Y_test, dtype=torch.long)

samples, features = X_train.shape
classes = Y_test.unique()

print(features)
print(samples)
print(X_train.shape, X_test.shape, Y_train.shape, Y_test.shape)
print(X_train)


In [ ]:
mean = X_train.mean(axis=0)
std = X_train.std(axis=0)
X_train = (X_train - mean)/ std
X_test = (X_test - mean)/ std
print(X_train)
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                        

In [ ]:
from torch import nn

class Classifier(nn.Module):
    def __init__(self):
        super(Classifier, self).__init__()
        self.first_layer = nn.Linear(features, 5)
        self.second_layer = nn.Linear(5, 10)
        self.third_layer = nn.Linear(10, 15)
        self.fourth_layer=nn.Linear(15,20)
        self.fifth_layer=nn.Linear(20,25)
        self.final_layer = nn.Linear(25,3)
        self.relu = nn.ReLU()
        self.softmax = nn.Softmax(dim=1)

    def forward(self, X_batch):
        layer_out = self.relu(self.first_layer(X_batch))
        print(layer_out)
        layer_out = self.relu(self.second_layer(layer_out))
        layer_out = self.relu(self.third_layer(layer_out))
        layer_out = self.relu(self.fourth_layer(layer_out))
        layer_out = self.relu(self.fifth_layer(layer_out))
        

        return self.softmax(self.final_layer(layer_out))
classifier = Classifier()
preds = classifier(X_train)
print(preds)

def TrainModel(model, loss_func, optimizer, X, Y, epochs=500):
    l=[]
    for i in range(epochs):
        preds = model(X) ## Make Predictions by forward pass through network
        loss = loss_func(preds, Y) ## Calculate Loss
        optimizer.zero_grad() ## Zero weights before calculating gradients
        loss.backward() ## Calculate Gradients
        optimizer.step() ## Update Weights
        if i % 100 == 0: ## Print MSE every 100 epochs
            print("NegLogLoss : {:.2f}".format(loss))

In [ ]:
from torch.optim import SGD
from sklearn.metrics import accuracy_score

torch.manual_seed(42) ##For reproducibility.This will make sure that same random weights are initialized each time.
epochs = 1500
learning_rate = torch.tensor(0.01) # 0.01 1/1e2

classifier = Classifier()
nll_loss = nn.NLLLoss()#negative log likelyhood
optimizer = SGD(params=classifier.parameters(), lr=learning_rate)#sgd-stochasitic gradient descend optimizer
accuracy=[]
for i in range(100,1000,100):
    TrainModel(classifier, nll_loss, optimizer, X_train, Y_train, epochs=i)

    test_preds = classifier(X_test) ## Make Predictions on test dataset

    test_preds = torch.argmax(test_preds, axis=1) ## Convert Probabilities to class type
    accuracy.append(format(accuracy_score(Y_test, test_preds)))
import matplotlib.pyplot as plt
plt.plot(range(100,1000,100),accuracy)
plt.xlabel("Epochs")
plt.ylabel("Accuracy")
plt.show()

train_preds = classifier(X_train) ## Make Predictions on train dataset

train_preds = torch.argmax(train_preds, axis=1) ## Convert Probabilities to class type

test_preds, train_preds


In [ ]:
from sklearn.metrics import accuracy_score

print("Train Accuracy : {:.2f}".format(accuracy_score(Y_train, train_preds)))
print("Test  Accuracy : {:.2f}".format(accuracy_score(Y_test, test_preds)))

from sklearn.metrics import classification_report

print("Test Data Classification Report : ")
print(classification_report(Y_test, test_preds))